## 1️⃣ Setup Environment

In [ ]:
# Check GPU availability
!nvidia-smi --query-gpu=name,memory.total,compute_cap --format=csv

In [ ]:
# Install required packages
!pip install -q transformers accelerate bitsandbytes torch sentencepiece protobuf
!pip install -q huggingface_hub

print("✓ Packages installed!")

In [ ]:
# Mount Google Drive for saving results
from google.colab import drive
drive.mount('/content/drive')

# Create benchmark directory in Drive
import os
DRIVE_RESULTS_DIR = '/content/drive/MyDrive/llm_benchmark_results'
os.makedirs(DRIVE_RESULTS_DIR, exist_ok=True)
print(f"✓ Results will be saved to: {DRIVE_RESULTS_DIR}")

In [ ]:
# (Optional) Login to HuggingFace for gated models (Llama, etc.)
from huggingface_hub import login

# Get your token from: https://huggingface.co/settings/tokens
# Uncomment and run if you need access to gated models:
# login(token="your_hf_token_here")

## 2️⃣ Define TransformersBackend

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from typing import List, Dict, Any, Optional
import time

class TransformersBackend:
    """
    HuggingFace Transformers backend optimized for T4 GPU.
    """
    
    def __init__(
        self,
        device: str = "cuda",
        use_4bit: bool = False,
        max_new_tokens: int = 512
    ):
        self.device = device
        self.use_4bit = use_4bit
        self.max_new_tokens = max_new_tokens
        self.model = None
        self.tokenizer = None
        self.current_model_name = None
        
        # Model mappings (Ollama-style -> HuggingFace)
        self.model_mappings = {
            # Qwen models (good for T4)
            "qwen2.5:0.5b": "Qwen/Qwen2.5-0.5B-Instruct",
            "qwen2.5:1.5b": "Qwen/Qwen2.5-1.5B-Instruct",
            "qwen2.5:3b": "Qwen/Qwen2.5-3B-Instruct",
            "qwen2.5:7b": "Qwen/Qwen2.5-7B-Instruct",
            # Gemma models
            "gemma2:2b": "google/gemma-2-2b-it",
            # Phi models
            "phi3.5:3.8b": "microsoft/Phi-3.5-mini-instruct",
            # SmolLM models (very efficient)
            "smollm2:135m": "HuggingFaceTB/SmolLM2-135M-Instruct",
            "smollm2:360m": "HuggingFaceTB/SmolLM2-360M-Instruct",
            "smollm2:1.7b": "HuggingFaceTB/SmolLM2-1.7B-Instruct",
            # Llama models (require HF login)
            "llama3.2:1b": "meta-llama/Llama-3.2-1B-Instruct",
            "llama3.2:3b": "meta-llama/Llama-3.2-3B-Instruct",
        }
    
    def get_hf_name(self, model_name: str) -> str:
        """Convert to HuggingFace model name."""
        return self.model_mappings.get(model_name, model_name)
    
    def load_model(self, model_name: str):
        """Load model with T4 optimizations."""
        hf_name = self.get_hf_name(model_name)
        
        if self.current_model_name == model_name:
            return  # Already loaded
        
        # Clear previous model
        if self.model is not None:
            del self.model
            del self.tokenizer
            torch.cuda.empty_cache()
        
        print(f"Loading model: {hf_name}")
        
        # Load tokenizer
        self.tokenizer = AutoTokenizer.from_pretrained(hf_name, trust_remote_code=True)
        if self.tokenizer.pad_token is None:
            self.tokenizer.pad_token = self.tokenizer.eos_token
        
        # Prepare model kwargs
        model_kwargs = {
            "torch_dtype": torch.float16,
            "device_map": "auto",
            "trust_remote_code": True,
        }
        
        if self.use_4bit:
            model_kwargs["quantization_config"] = BitsAndBytesConfig(
                load_in_4bit=True,
                bnb_4bit_compute_dtype=torch.float16,
                bnb_4bit_quant_type="nf4",
                bnb_4bit_use_double_quant=True
            )
        
        self.model = AutoModelForCausalLM.from_pretrained(hf_name, **model_kwargs)
        self.model.eval()
        self.current_model_name = model_name
        
        # Print memory usage
        allocated = torch.cuda.memory_allocated() / 1e9
        reserved = torch.cuda.memory_reserved() / 1e9
        print(f"✓ Model loaded. VRAM: {allocated:.2f}GB allocated, {reserved:.2f}GB reserved")
    
    def generate(self, model_name: str, prompt: str) -> Dict[str, Any]:
        """Generate response."""
        start_time = time.time()
        
        try:
            self.load_model(model_name)
            
            inputs = self.tokenizer(
                prompt,
                return_tensors="pt",
                padding=True,
                truncation=True,
                max_length=2048
            ).to(self.model.device)
            
            with torch.no_grad():
                outputs = self.model.generate(
                    **inputs,
                    max_new_tokens=self.max_new_tokens,
                    do_sample=True,
                    temperature=0.7,
                    top_p=0.9,
                    pad_token_id=self.tokenizer.pad_token_id,
                )
            
            # Decode new tokens only
            input_len = inputs['input_ids'].shape[1]
            response = self.tokenizer.decode(
                outputs[0][input_len:],
                skip_special_tokens=True
            ).strip()
            
            return {
                "response": response,
                "success": True,
                "error_type": None,
                "duration": round(time.time() - start_time, 3)
            }
            
        except Exception as e:
            return {
                "response": f"ERROR: {str(e)}",
                "success": False,
                "error_type": "TRANSFORMERS_ERROR",
                "duration": round(time.time() - start_time, 3)
            }
    
    def get_available_models(self) -> List[str]:
        return list(self.model_mappings.keys())
    
    def clear_cache(self):
        if self.model is not None:
            del self.model
            del self.tokenizer
            self.model = None
            self.tokenizer = None
            self.current_model_name = None
            torch.cuda.empty_cache()
            print("✓ Cache cleared")

# Initialize backend
USE_4BIT = False  # @param {type:"boolean"}
MAX_TOKENS = 512  # @param {type:"integer"}

backend = TransformersBackend(use_4bit=USE_4BIT, max_new_tokens=MAX_TOKENS)
print(f"\n✓ Backend initialized (4-bit: {USE_4BIT}, max_tokens: {MAX_TOKENS})")
print(f"Available models: {backend.get_available_models()}")

## 3️⃣ Load Benchmark Data

In [ ]:
# Clone or upload benchmark data
# Option 1: Clone from GitHub (replace with your repo)
# !git clone https://github.com/axttt243567/llm-benchmark.git

# Option 2: Upload data manually
# from google.colab import files
# uploaded = files.upload()  # Upload your JSON dataset files

# Option 3: Use sample data (for testing)
import json

# Sample benchmark questions
SAMPLE_QUESTIONS = [
    {
        "id": "COMMON_001",
        "domain": "common_sense",
        "category": "everyday",
        "difficulty": "easy",
        "test_for": "basic reasoning",
        "prompt": "If it's raining outside and you want to stay dry, what should you bring with you?"
    },
    {
        "id": "LOGIC_001",
        "domain": "logic",
        "category": "tricky",
        "difficulty": "medium",
        "test_for": "logical reasoning",
        "prompt": "A farmer has 17 sheep. All but 9 run away. How many sheep does the farmer have left?"
    },
    {
        "id": "MATH_001",
        "domain": "math",
        "category": "algebra",
        "difficulty": "medium",
        "test_for": "algebraic solving",
        "prompt": "Solve for x: 2x + 5 = 13"
    },
    {
        "id": "COMMON_002",
        "domain": "common_sense",
        "category": "social",
        "difficulty": "easy",
        "test_for": "social understanding",
        "prompt": "Your friend just received bad news about a job application. What would be an appropriate thing to say?"
    },
    {
        "id": "LOGIC_002",
        "domain": "logic",
        "category": "tricky",
        "difficulty": "hard",
        "test_for": "attention to detail",
        "prompt": "How many times can you subtract 5 from 25?"
    }
]

print(f"✓ Loaded {len(SAMPLE_QUESTIONS)} sample questions")
print("\nTo use your own data, upload JSON files or clone your benchmark repo.")

## 4️⃣ Run Benchmark

In [ ]:
import datetime
import hashlib
from tqdm.notebook import tqdm

def run_benchmark(model_name: str, questions: list, dataset_key: str = "custom"):
    """
    Run benchmark on the given model and questions.
    """
    run_id = datetime.datetime.now().strftime("%Y%m%d_%H%M%S") + "_" + hashlib.md5(str(time.time()).encode()).hexdigest()[:6]
    
    print(f"\n{'='*60}")
    print(f"🚀 Starting Benchmark")
    print(f"{'='*60}")
    print(f"Model: {model_name}")
    print(f"Dataset: {dataset_key}")
    print(f"Questions: {len(questions)}")
    print(f"Run ID: {run_id}")
    print(f"{'='*60}\n")
    
    results = []
    total_time = 0
    successful = 0
    
    for i, q in enumerate(tqdm(questions, desc="Processing")):
        prompt = q.get('prompt', '')
        
        # Generate response
        result = backend.generate(model_name, prompt)
        
        # Track stats
        total_time += result['duration']
        if result['success']:
            successful += 1
        
        # Build result entry
        entry = {
            "run_id": run_id,
            "question_index": i + 1,
            "question_id": q.get('id', f'Q_{i+1}'),
            "domain": q.get('domain', ''),
            "category": q.get('category', ''),
            "difficulty": q.get('difficulty', 'medium'),
            "test_for": q.get('test_for', ''),
            "prompt": prompt,
            "model_tested": model_name,
            "timestamp": datetime.datetime.now().isoformat(),
            "response": result['response'],
            "metrics": {
                "response_time_seconds": result['duration'],
                "response_length_chars": len(result['response']),
                "response_length_words": len(result['response'].split())
            },
            "execution_success": result['success'],
            "error_type": result['error_type']
        }
        results.append(entry)
        
        # Show progress
        if result['success']:
            preview = result['response'][:100] + "..." if len(result['response']) > 100 else result['response']
            print(f"✓ [{i+1}/{len(questions)}] {q.get('id', 'Q')} ({result['duration']:.2f}s)")
        else:
            print(f"✗ [{i+1}/{len(questions)}] {q.get('id', 'Q')} - {result['error_type']}")
    
    # Build final output
    hf_model = backend.get_hf_name(model_name)
    output = {
        "metadata": {
            "run_id": run_id,
            "model": model_name,
            "hf_model": hf_model,
            "backend": {
                "type": "transformers",
                "device": "T4 GPU",
                "use_4bit": backend.use_4bit
            },
            "dataset": {
                "key": dataset_key,
                "total_questions": len(questions)
            },
            "execution": {
                "platform": "Google Colab",
                "start_time": results[0]['timestamp'] if results else None,
                "end_time": datetime.datetime.now().isoformat(),
            },
            "statistics": {
                "total_questions": len(questions),
                "successful": successful,
                "failed": len(questions) - successful,
                "success_rate": round(successful / len(questions) * 100, 2) if questions else 0,
                "total_response_time": round(total_time, 2),
                "average_response_time": round(total_time / len(questions), 3) if questions else 0
            }
        },
        "results": results
    }
    
    # Save to Drive
    clean_model = model_name.replace(':', '_').replace('/', '_')
    filename = f"{dataset_key}_{clean_model}_{run_id}.json"
    filepath = os.path.join(DRIVE_RESULTS_DIR, filename)
    
    with open(filepath, 'w', encoding='utf-8') as f:
        json.dump(output, f, indent=2, ensure_ascii=False)
    
    # Print summary
    print(f"\n{'='*60}")
    print(f"✅ BENCHMARK COMPLETE")
    print(f"{'='*60}")
    print(f"Model: {model_name}")
    print(f"Questions: {len(questions)}")
    print(f"Success Rate: {output['metadata']['statistics']['success_rate']}%")
    print(f"Total Time: {total_time:.2f}s")
    print(f"Avg Time: {output['metadata']['statistics']['average_response_time']:.3f}s")
    print(f"\n📁 Results saved to: {filepath}")
    print(f"{'='*60}\n")
    
    return output

print("✓ Benchmark function defined")

In [ ]:
# Select model and run benchmark
# @title Select Model and Run
MODEL = "qwen2.5:1.5b"  # @param ["qwen2.5:0.5b", "qwen2.5:1.5b", "qwen2.5:3b", "smollm2:360m", "smollm2:1.7b", "phi3.5:3.8b", "gemma2:2b"]
DATASET_NAME = "sample_test"  # @param {type:"string"}

# Run the benchmark
results = run_benchmark(MODEL, SAMPLE_QUESTIONS, DATASET_NAME)

## 5️⃣ View Results

In [ ]:
# Display results in a nice format
import pandas as pd

if 'results' in dir() and results:
    df = pd.DataFrame([
        {
            'ID': r['question_id'],
            'Domain': r['domain'],
            'Time (s)': r['metrics']['response_time_seconds'],
            'Words': r['metrics']['response_length_words'],
            'Success': '✓' if r['execution_success'] else '✗',
            'Response Preview': r['response'][:80] + '...' if len(r['response']) > 80 else r['response']
        }
        for r in results['results']
    ])
    display(df)
else:
    print("No results to display. Run a benchmark first.")

In [ ]:
# View individual responses
# @title View Response Details
QUESTION_INDEX = 0  # @param {type:"integer"}

if 'results' in dir() and results and QUESTION_INDEX < len(results['results']):
    r = results['results'][QUESTION_INDEX]
    print(f"Question ID: {r['question_id']}")
    print(f"Domain: {r['domain']} / {r['category']}")
    print(f"Difficulty: {r['difficulty']}")
    print(f"\n📝 Prompt:\n{r['prompt']}")
    print(f"\n🤖 Response:\n{r['response']}")
    print(f"\n⏱️ Time: {r['metrics']['response_time_seconds']:.3f}s")
else:
    print("Invalid index or no results available.")

## 6️⃣ Cleanup

In [ ]:
# Clear GPU memory when done or before loading a different model
backend.clear_cache()

# Show memory status
if torch.cuda.is_available():
    allocated = torch.cuda.memory_allocated() / 1e9
    reserved = torch.cuda.memory_reserved() / 1e9
    print(f"VRAM: {allocated:.2f}GB allocated, {reserved:.2f}GB reserved")

## 📋 Tips for T4 GPU

1. **Model Size**: T4 has 16GB VRAM. Stick to models ≤7B for smooth operation.

2. **4-bit Quantization**: Enable `USE_4BIT = True` to run larger models (up to 13B).

3. **Clear Cache**: Use `backend.clear_cache()` before loading a new model.

4. **Gated Models**: For Llama models, you need to:
   - Accept the license at huggingface.co/meta-llama
   - Login with `login(token="your_token")`

5. **Custom Models**: Add your own model mappings:
   ```python
   backend.model_mappings["my_model"] = "org/model-name"
   ```

6. **Session Limits**: Colab free tier has usage limits. Save results to Drive!